In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from zipfile import ZipFile

import datetime
import json

In [2]:
pathname = "/home/benjamin/Folders_Python/Weight/imports/"

polar_filename = pathname + "TBD.zip"  # données de Polar

# next : aller chercher les archives zip les plus récentes commençant par data_BEN et File-Export...

In [3]:
# Récupère données POIDS

# Source : Withings https://healthmate.withings.com/

# Account > User > Parameters > Download ==> data_BEN_xxxxx.zip ==> contient weight.csv

withings_filename = pathname + "data_BEN.zip"  # données de HealthMate

with ZipFile(withings_filename, 'r') as weight_zip:
    weight_csv = weight_zip.extract('weight.csv', path=pathname)

# extrait la dataframe poids ----------------

colnames = ['Date', 'Poids (kg)', 'Gras (kg)']

df_weight = pd.read_csv(weight_csv, usecols=colnames)
df_weight.rename(columns = {'Poids (kg)' : 'Masse_Totale' , 'Gras (kg)' : 'Masse_Grasse'}, inplace=True)

# transforme le champ str de Date en datetime object

date_format = '%Y-%m-%d %H:%M:%S'

df_weight['Date'] = df_weight['Date'].apply(lambda x : datetime.datetime.strptime(x, date_format).date())
df_weight = df_weight.groupby('Date').mean()

start_date = datetime.date(2020, 8, 1)  # on prend les data depuis le 1er Août 2020

df_weight = df_weight[df_weight.index >= start_date]
df_weight.dropna(inplace=True)

df_weight['Masse_Maigre'] = df_weight['Masse_Totale'] - df_weight['Masse_Grasse']
df_weight['BMR'] = 370 + 21.6 * df_weight['Masse_Maigre']

In [4]:
df_weight.tail(10)

,Masse_Totale,Masse_Grasse,Masse_Maigre,BMR
Date,,,,
2022-12-24,72.500000,10.766000,61.734000,1703.454400
2022-12-25,72.478571,11.037143,61.441429,1697.134857
2022-12-26,71.950000,10.984286,60.965714,1686.859429
2022-12-27,72.680000,11.192000,61.488000,1698.140800
2022-12-28,72.018750,11.908750,60.110000,1668.376000
2022-12-29,71.566667,11.422222,60.144444,1669.120000
2022-12-30,71.310000,11.210000,60.100000,1668.160000
2022-12-31,71.810000,10.890000,60.920000,1685.872000
2023-01-01,71.800000,9.500000,62.300000,1715.680000


In [5]:
# Récupère données FOOD

# https://www.myfitnesspal.com/reports

# www.myfitnesspal.com ==> reports > export data ==> File-Export-date1-to-date2.zip

mfp_filename = pathname + "File-Export.zip" # données de My Fitness Pal

# Extrait les archives
    
with ZipFile(mfp_filename, 'r') as food_zip:
    output_dir = food_zip.namelist()
    target = 'Nutrition-Summary'
    for l in output_dir:
        if l[:len(target)] == target:
            food_csv = food_zip.extract(l, path=pathname)
            break
        
# extrait la dataframe food -------------------------------

colnames = ['Date', 'Meal', 'Calories', 'Fat (g)', 'Carbohydrates (g)', 'Protein (g)']

df_food = pd.read_csv(food_csv, usecols=colnames)
df_food.rename(columns = {'Fat (g)' : 'Lipides' , 'Carbohydrates (g)' : 'Glucides', 'Protein (g)' : 'Proteines'}, inplace=True)

# transforme le champ str de Date en datetime object

date_format = '%Y-%m-%d'

df_food['Date'] = df_food['Date'].apply(lambda x : datetime.datetime.strptime(x, date_format).date())
df_food = df_food.groupby('Date').sum()

start_date = datetime.date(2020, 8, 1)  # on prend les data depuis le 1er Août 2020

df_food = df_food[df_food.index >= start_date]
df_food.dropna(inplace=True)

In [6]:
df_food.tail(10)

,Calories,Lipides,Glucides,Proteines
Date,,,,
2022-12-25,1850.3,34.7,166.0,160.7
2022-12-26,2328.3,58.3,273.1,173.1
2022-12-27,1611.4,37.5,120.1,189.5
2022-12-28,1767.4,39.0,180.2,166.6
2022-12-29,1784.0,33.7,179.0,189.0
2022-12-30,1666.7,36.4,141.3,191.9
2022-12-31,3197.2,66.4,424.4,203.1
2023-01-01,1726.4,38.4,152.0,195.1
2023-01-02,2044.6,47.1,233.7,173.5


In [7]:
# Récupère les données EXERCICE

# https://account.polar.com/#export ==> polar-user-data-export-xxxxx.zip

polar_filename = pathname + "polar-user-data-export.zip"  # données de HealthMate

with ZipFile(polar_filename, 'r') as polar_zip:
    output_dir = polar_zip.namelist()
    
    target = 'training-session'
    for enr_name in output_dir: # on parcourt la liste des archives
        if enr_name[:len(target)] == target:  # si c'est un enregistrement d'une session de training, on traite
            # enr_json = polar_zip.extract(enr_name, path=pathname)   # extraction du json
            # os.remove(enr_json)
            pass


# # extrait la dataframe exercices -------------------------------

# colnames = ['Début', 'Fin', "Type d'activité", 'Données']

# df_exercices = pd.read_csv(exercice_csv, usecols=colnames)
# df_exercices.rename(columns = {'Début' : 'Debut' , "Type d'activité" : "Type", "Données" : "Donnees" }, inplace=True)

# # transforme le champ str de Date en datetime object

# date_format = '%Y-%m-%dT'

# # df_food['Date'] = df_food['Date'].apply(lambda x : datetime.datetime.strptime(x, date_format).date())
# # df_food = df_food.groupby('Date').sum()

# # start_date = datetime.date(2020, 8, 1)  # on prend les data depuis le 1er Août 2020

# # df_food = df_food[df_food.index >= start_date]
# # df_food.dropna(inplace=True)

In [8]:
testname = "/home/benjamin/Folders_Python/Weight/imports/training-session.json"

with open(testname, 'r') as f:
    exo_dict = json.load(f)
    print(type(exo_dict))

<class 'dict'>


In [9]:
exo_dict

{'exportVersion': '1.6',
 'name': '',
 'deviceId': 'A739AA25',
 'startTime': '2022-01-26T17:50:35.532',
 'stopTime': '2022-01-26T18:20:40.156',
 'timeZoneOffset': 60,
 'duration': 'PT1798.587S',
 'maximumHeartRate': 138,
 'averageHeartRate': 99,
 'kiloCalories': 235,
 'physicalInformationSnapshot': {'dateTime': '2022-01-26T17:21:45.000',
  'sex': 'MALE',
  'birthday': '1968-11-02',
  'height, cm': 181.0,
  'weight, kg': 69.0,
  'vo2Max': 66,
  'maximumHeartRate': 177,
  'restingHeartRate': 50,
  'aerobicThreshold': 133,
  'anaerobicThreshold': 159,
  'sleepGoal': 'PT21600S',
  'functionalThresholdPower': 294},
 'exercises': [{'startTime': '2022-01-26T17:50:35.696',
   'stopTime': '2022-01-26T18:20:34.283',
   'timezoneOffset': 60,
   'duration': 'PT1798.587S',
   'sport': 'INDOOR_ROWING',
   'kiloCalories': 235,
   'heartRate': {'min': 59, 'avg': 99, 'max': 138},
   'zones': {'heart_rate': [{'lowerLimit': 89,
      'higherLimit': 106,
      'inZone': 'PT996S',
      'zoneIndex': 1},
  